In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from scipy.stats import gmean
from xbbg import blp
import sys
sys.path.append(r'\\10.5.20.31\交易部\Blair\backTestTool')
sys.path.append(r'\\10.5.20.31\交易部\FX trading Strategy souce code\Wilson\程式碼')
import my_function as my
import backtestModuleVer2_1128
from backtestModuleSteveVer import (DrawDown, calculate_detailed_mdd, generate_recent_returns_df, 
                                    print_stats, visualize_portfolio, format_position, 
                                    calculate_portfolio_value,process_portfolio_data)

import sys
from contextlib import contextmanager
import io
import os

# Add this at the beginning of your code, after the imports
strgyName = '1105'
traderName = 'Blair'


devCcy =  ['EUR', 'JPY', 'GBP', 'CHF', 'AUD', 'CAD', 'NZD', 'SEK']
revL = ['NZD', 'EUR', 'AUD', 'GBP']
basket = devCcy
#########################################
from datetime import datetime
today = datetime.now()+pd.Timedelta(days=0)
today_str = today.strftime('%Y-%m-%d')
# startDate = today-pd.Timedelta(days=600)
startDate = '2012-01-01'

# startDate = startDate.strftime('%Y-%m-%d')
endDate = today_str

dr = pd.date_range(
    start=pd.to_datetime(startDate),
    end=pd.to_datetime(endDate),
    freq=BDay()
)  

spot = pd.DataFrame(index=dr, columns=basket)
info = pd.read_excel(r'\\10.5.20.31\交易部\Blair\backTestTool\ccyInfo.xlsx')
info = info[info['ccy'].isin(basket)]
info.index = info['ccy']

for i in basket:
    spot[i] = blp.bdh(tickers=info.loc[i, 'bbgSpotTicker'], flds = ["PX_LAST"], start_date=startDate, end_date=endDate).reindex(dr).ffill()

df = pd.DataFrame(index=dr)
df['close'] = spot['JPY']

gotobiL = [5, 10, 15, 20, 25, 30]
gotobiD = dict.fromkeys(gotobiL)

filterDf = pd.DataFrame(index=dr)

dxy = blp.bdh(tickers='JPY CMTK Curncy', flds = ["PX_LAST"], start_date=startDate, end_date=endDate)
dxy.index = pd.DatetimeIndex(dxy.index)
dxy = dxy.reindex(dr).ffill()

filterDf['dxy'] = dxy

filterDf['USDJPY_MA_fast'] = filterDf['dxy'].rolling(window=40).mean()
filterDf['USDJPY_MA_slow'] = filterDf['dxy'].rolling(window=100).mean()


notGoodDay = filterDf[filterDf['USDJPY_MA_fast'] < filterDf['USDJPY_MA_slow']].index 
signalDf = pd.DataFrame(0, index=dr, columns=basket) 
for targetDay in gotobiL:
    signalDf.loc[signalDf.index.day == (targetDay-1), 'JPY'] = 1
    signalDf.loc[signalDf.index.day == (targetDay), 'JPY'] = 0

signalDf[signalDf.index.isin(notGoodDay)] = 0

weight_size = 2
signalDf = signalDf * weight_size   
signalDf.to_csv(f'{strgyName}_gotobi_Signal.csv')

# Wilson 測試區~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
position_final = signalDf.copy()
print(f'以下為 Gotobi ({strgyName}) 策略: ')
print("本月部位狀況: ")
print(position_final.iloc[-5:, :])

def format_trade_list(condition_summary):
    trade_list = (condition_summary.iloc[-1,] - condition_summary.iloc[-2,])
    # trade_list = (condition_summary.iloc[-1,]) *-1
    formatted_positions = []
    for i in condition_summary.columns:        
        position_value = trade_list[i]
        if position_value == 0:
            continue
        
        
        position_direction = 'Long' if position_value > 0 else 'Short'

        exchange_rate = None
        # If the currency is one of the specified ones, ask for the exchange rate
        if i in ['EUR', 'GBP', 'AUD', 'NZD']:
            exchange_rate = float(input(f"Please enter the exchange rate for {i}USD: "))
            position_value /= exchange_rate

        # Format the currency pair based on the given currency
        currency_pair = f"{i}USD" if i in ['EUR', 'GBP', 'AUD', 'NZD'] else f"USD{i}"

        # Prepare the formatted output string
        mio_suffix = f"mio in {i} ({exchange_rate})" if exchange_rate else "mio"
        position_output = f"{position_direction} {currency_pair} {abs(position_value):.2f}{mio_suffix}"
        formatted_positions.append(position_output)
    
    return formatted_positions


#########################################################################################################################################
# 輸出的部分
print("\n")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Wilson 指令輸出區")
print("還在測試中，請仔細檢查")
# 使用函數
formatted_positions = format_trade_list(position_final)
for position in formatted_positions:
    print(position if position else 'None')

#########################################################################################################################################
# 處理信號
# start_date = '2024-01-01'
start_date = '2012-01-01'
weight_size = 2
signalDf = position_final.copy()
signalDf['Date'] = signalDf.index

last_date = signalDf["Date"].iloc[-1]
bt = backtestModuleVer2_1128.backTest(MIO=weight_size, signalDf=signalDf, startDate=start_date, endDate=last_date)
bt.backtestFun()
bt.strgyStats()

teStart = start_date
bt.portValChange[bt.portValChange.index >= teStart].to_csv(f'dailyRet_{strgyName}.csv')

tmpStart = '2025-01-01'
bt.portValChange[bt.portValChange.index >= tmpStart].to_csv(f'2025_dailyRet_{strgyName}.csv')

# 先處理存儲資料的部分
daily_total = round((bt.portChange).to_frame(name="Portfolio_Return_Profit"),5)
daily_total["Portfolio_Return_Profit"] = daily_total["Portfolio_Return_Profit"]*weight_size*1_000_000
daily_total = daily_total.loc[:today]
my.update_excel_save_excel(1105, daily_total)

daily_total = round((bt.portChange).to_frame(),5)
daily_total.columns = ["GDP_MoM"]
daily_total["GDP_MoM"] = daily_total["GDP_MoM"]*weight_size*1_000_000
daily_total_2024 = daily_total.loc["2024-01-02":today]

# file_path = r'\\10.5.20.31\交易部\1.周會資料\Tool\FOR0045彙整_Backup.xlsm'
# mapping_portfolio = {"BKFXOP07DB06TA": "IMM"}
# df_07 = process_portfolio_data(file_path, mapping_portfolio)

# 打印统计信息
print_stats(daily_total_2024['GDP_MoM'], n_days=10)

# print("\n")
# visualize_portfolio(daily_total_2024, df_07, 'IMM', 'IMM', start_date, "IMM")
# print("\n")